<a href="https://colab.research.google.com/github/JPalacios-8/DL-Proyecto-1/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install medmnist
import medmnist
from medmnist import INFO, Evaluator
from medmnist import ChestMNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Confirmar que estás usando la versión 2D en resolución 28x28
data_flag = 'chestmnist'
download = True


# Obtener información del dataset
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

# Transforms para imágenes en tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# Cargar datos de entrenamiento y prueba
train_dataset = DataClass(split='train', transform=transform, download=download)
test_dataset = DataClass(split='test', transform=transform, download=download)
val_dataset = DataClass(split='val', transform=transform, download=download)

# DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)


100%|██████████| 82.8M/82.8M [00:07<00:00, 11.7MB/s]


In [3]:
# Comprobación de que es 28x28
image, label = train_dataset[0]
print("Forma de la imagen:", image.shape)


Forma de la imagen: torch.Size([1, 28, 28])
